In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!rm -rf dsnlplib data
!git clone https://github.com/marcocuccarini/structuralBioInformatic.git; mv strucuralBioInformatic/* .; rm -rf strucuralBioInformatic;

Cloning into 'structuralBioInformatic'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 59 (delta 25), reused 49 (delta 15), pack-reused 0
Unpacking objects: 100% (59/59), 11.11 KiB | 214.00 KiB/s, done.
mv: cannot stat 'strucuralBioInformatic/*': No such file or directory


In [3]:
#Set path
path = 'drive/MyDrive/SB_practical_notebooks/data/'

In [4]:
!cp drive/MyDrive/SB_practical_notebooks/data/calc_features.py
!pip install imbalanced-learn

cp: missing destination file operand after 'drive/MyDrive/SB_practical_notebooks/data/calc_features.py'
Try 'cp --help' for more information.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from structuralBioInformatic import LibraryStructural
from imblearn.under_sampling import NearMiss

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn import preprocessing
from sklearn import svm, datasets

from keras.layers import Dense
from keras.layers import Dropout

from keras.models import Sequential
from keras.utils import np_utils


import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import table

import math
import pandas as pd
import os
import random
import numpy as np
from numpy import nan
from numpy import where
from itertools import cycle

from google.colab import files
from collections import Counter
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot


from tensorflow.keras import regularizers
import tensorflow as tf


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [6]:
consider_unclassified=0 #decide if we wont consider unkown class or delete all te row with unkown

# Dataset definition

In [7]:
#get the data from csv file
dataSet = pd.read_csv(path + 'SB_dataset2.csv')
#We pass 1 if we wont consider "Unknown" like a class, 0 if we wont drop the unkown example
df=LibraryStructural.DatasetDefinition(consider_unclassified,dataSet)

FileNotFoundError: ignored

In [ ]:
#Get the number of sample for each class, iuse it for the paramethere weights_class of the NN
class_size=df.groupby('Interaction')['s_ss8'].count().reset_index(name='#')    

Get Disatance between ['t_a1','t_a2','t_a3','t_a4','t_a5'] and ['s_a1','s_a2','s_a3','s_a4','s_a5']

In [ ]:
df1=df[['t_a1','t_a2','t_a3','t_a4','t_a5']]
df2=df[['s_a1','s_a2','s_a3','s_a4','s_a5']]
num=df1.to_numpy()
num2=df2.to_numpy()
listist=[]
for i in range(len(num)):
  listist.append(np.linalg.norm(num[i]-num2[i]))
df['dist']=listist

In [ ]:
df=df.dropna()

In [ ]:
#I Splt the dataset in features and label
df,y,label=LibraryStructural.Get_Label(df)

In [ ]:
#df.reset_index(inplace=True)

In [ ]:
for i in label:
  listTot=[]
  
  listT=(df[(y==i)]['t_resn']).to_list()
  listP=(df[(y==i)]['s_resn']).to_list()
  print(i)
  for i in range(len(listT)):
    listTot.append(listT[i]+"->"+listP[i])
  listCont=[]
  pair=list(set(listTot))
  
  for h in pair:
    listCont.append(float("{:.4f}".format((listTot.count(h))/len(listTot))))
  sort=[pair for _, pair in sorted(zip(listCont, pair))]
  


  print(sort)
  print(sorted(listCont, reverse=True))
  

  

# Preprocessing

In [ ]:
df=LibraryStructural.Preprocessing(df)

#Labels encoding 

In [ ]:
df,y,y_bin=LibraryStructural.LabelEncoder(df,y)

SplitTheDataset

In [ ]:
X_train, X_test, y_train, y_test, ybin_train, ybin_test = train_test_split(df, y, y_bin, test_size = 0.33, random_state = 42)

The function "OverSample" try to fix the problem of the unbelancedd classes creating from the class with a low number of example new datas.

# NAIVE BAYES


# NAIVE BAYES + Class join


In [ ]:
#For join the two class i change all the label==5 in 0

y_train_j=[]
for i in y_train:
  if (i==5):
    y_train_j.append(0)
  else:
    y_train_j.append(i)

y_test_j=[]
for i in y_test:
  if (i==5):
    y_test_j.append(0)
  else:
    y_test_j.append(i)


In [ ]:
#I define the model
gnb = GaussianNB()
model = gnb.fit(X_train, y_train_j)
y_pred_NB=model.predict(X_test)

In [ ]:
set(y_pred_NB)

In [ ]:
labelJ=['SSBOND/PICATION', 'HBOND', 'PIPISTACK', 'VDW', 'IONIC']

In [ ]:
print(classification_report(y_test_j, y_pred_NB,target_names=labelJ))

In [ ]:
cm = confusion_matrix(y_test_j, y_pred_NB)
cmn = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(14,14))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=labelJ, yticklabels=labelJ ,cmap=plt.cm.Blues)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

Create a model for classify the two different class

In [ ]:
import copy

In [ ]:
X_train1=copy.deepcopy(X_train)
X_test1=copy.deepcopy(X_test)


In [ ]:
X_train1['y']=y_train
X_test1['y']=y_test


In [ ]:
X_train1 = X_train1[X_train1['y'].isin([0,5])]
X_test1 = X_test1[X_test1['y'].isin([0,5])]

In [ ]:
y_train1=X_train1['y']
y_test1=X_test1['y']
del X_train1['y']
del X_test1['y']

In [ ]:
y_tr=[]
for i in y_train1:
  if(i==5):
    y_tr.append(1)
  else:
    y_tr.append(0)
y_te=[]
for i in y_test1:
  if(i==5):
    y_te.append(1)
  else:
    y_te.append(0) 

In [ ]:
len(y_te)

In [ ]:
yy = preprocessing.LabelEncoder()
yy.fit(y_tr)
y=yy.transform(y_tr)
y_bin_tr = np_utils.to_categorical(y)

In [ ]:
X_trainCut=X_train1[['t_resn','t_resn','dist']]
X_testCut=X_test1[['t_resn','t_resn','dist']]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
weigth={0:1, 1:2}
hist = model.fit(X_train, y_bin_tr, epochs=50,validation_split=0.2, callbacks=[callback], class_weight=weigth)


# Fit on training data
model.fit(X_trainCut, y_bin_tr)

In [ ]:
ybin_pred_NN = model.predict(X_testCut)

In [ ]:
ybin_pred_NN

In [ ]:
len(ybin_pred_NN)

In [ ]:
print(classification_report(y_te, np.argmax(ybin_pred_NN, axis=1), target_names=['SSBOND','PICATION']))

In [ ]:
cm = confusion_matrix(y_te, np.argmax(ybin_pred_NN, axis=1))
cmn = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots(figsize=(14,14))
sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=['SSBOND','PICATION'], yticklabels=['SSBOND','PICATION'] ,cmap=plt.cm.Blues)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

#Join Performance of the 2 model.

In [ ]:
ybin_pred_NN

In [ ]:
np.argmax(ybin_pred_NN[0], axis=0)

In [ ]:
len(ybin_pred_NN)

In [ ]:
from collections import Counter

# Use a string as an argument
Counter(y_pred_NB)

In [ ]:
len(ybin_pred_NN)

In [ ]:
cont_NN=0
for i in range(len(y_pred_NB)):
  if y_pred_NB[i]==0:
   

    if np.argmax(ybin_pred_NN[cont_NN], axis=0)==1:
      y_pred_NB[i]=5
    cont_NN+=1

In [ ]:
print(classification_report(y_test, y_pred_NB))

In [ ]:
y_pred_NB

I get the prediction of a sigle value

In [ ]:
X_test.reset_index(inplace=True)

In [ ]:
del X_test['index']

In [ ]:
X_test.columns

In [ ]:
pred=gnb.predict_proba(X_test.loc[0,:].to_numpy().reshape(1,-1))

In [ ]:
#Prediction ofnaive bayes
pred

In [ ]:
pred1=model.predict(X_test.loc[0,:].to_numpy().reshape(1,-1))

In [ ]:
#Prediction of neural network
pred1

In [ ]:
x= pred1[0][0]
pred[0][0] = x*pred1[0][0]
predprob=np.append(pred[0],x*pred1[0][1])

In [ ]:
predprob